In [1]:
!pip install pycaret
!pip install python-dotenv

  Obtaining dependency information for pycaret from https://files.pythonhosted.org/packages/d5/54/d575af389203fc27d6c6cf7d60c4e67fcabfda4bc8e84271c8a396bd4a03/pycaret-3.1.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 35.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 14.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 34.4 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.4/159.4 kB 26.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for imbalanced-learn>=0.8.1 from https://files.pythonhosted.org/packages/a3/9e/fbe60a768502af54563dcb59ca7856f5a8833b3ad5ada658922e1ab09b7f/imbalanced_learn-0.11.0-py3-none-any.whl.metadata
  Obtaining dependency information for category-encoders>=2.4.0 from https://files.pythonhosted.org/packages/7f/e5/79a62e5c9c9ddbfa9ff5222240d408c1eeea4e38741a0dc8343edc7ef1ec/catego

<h3>1. Introduction </h3>
This template notebook will give users (data scientists and data engineers) the opportunities to train and deploy regression machine learning models with ease, without having to write all of the code from scratch. <br> 
There are a few requirements for the user anyway, such as;
<li>the data location</li> 
<li>the target (dependent) variable in your dataset and</li>
<li>what kind of machine learning algorithm you will be performing. </li>

All of these values will be entered in terraform and will be automatically applied in the notebook. 

<h3>2. Imports</h3>

The libraries that are required for this model notebook are imported below 

In [1]:
import boto3, os, tarfile
from sagemaker import get_execution_role
from dotenv import load_dotenv
from load_data import load_data
from split_data import split_data
import importlib
from save_model_to_s3 import save_model_to_s3
from deploy_model_endpoint import deploy_model

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/lanre.bakare/Library/Application Support/sagemaker/config.yaml


<h3>3. Loading Data</h3>
Here the user is required to specify the location of the data that they will like to use for prediction. An helper function is used to load the data from S3. 

<em>Note: Your data needs to be an s3 bucket.</em>

In [2]:
# Variables Setup Stage
load_dotenv(".env")
role = get_execution_role()

# Env variables
data_location_s3 = os.getenv("data_location_s3")
algorithm_choice = os.getenv("algorithm_choice")
target = os.getenv("target")
endpoint_name = os.getenv("endpoint_name")
model_name = os.getenv("model_name")
data_location = 's3://{}'.format(data_location_s3)
pycaret_ecr_name = os.getenv("pycaret_ecr_name")
instance_type = os.getenv("instance_type")

print(data_location_s3, algorithm_choice, target, endpoint_name, model_name, data_location, pycaret_ecr_name, instance_type)

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/lanre.bakare/Library/Application Support/sagemaker/config.yaml
streaming-data-platform-ml-data/bakerloo.csv regression Bakerloo10 lanre-trial-endpoint lanre-test s3://streaming-data-platform-ml-data/bakerloo.csv mlops-pycaret-repo ml.m4.xlarge


<h3>4. Read and display a sample of data</h3>

In [3]:
# Load data from S3
df = load_data(data_location)
df.head() 

,Bakerloo,Bakerloo10,dayOfWeek,hour,minute
0,23,24.0,1,13,38
1,23,22.0,1,13,40
2,23,24.0,1,13,41
3,23,24.0,1,13,42
4,23,24.0,1,13,43


## Importing Pycaret

<h3>5. Data Exploration</h3>

In [4]:
# Split and shuffle data
train_data, test_data = split_data(df, shuffle=True)
print(train_data, test_data)

      Bakerloo  Bakerloo10  dayOfWeek  hour  minute
0           22        22.0          4    10      24
1           24        23.0          6    18      15
2           17        20.0          6    21      38
3           21        21.0          4    21      39
4           10        10.0          6     6      59
...        ...         ...        ...   ...     ...
5696        24        25.0          4     7      32
5697        24        24.0          6    19      52
5698        24        26.0          4     8      39
5699        27        25.0          2     7      17
5700        26        27.0          0     6      31

[5701 rows x 5 columns]       Bakerloo  Bakerloo10  dayOfWeek  hour  minute
5701        22        18.0          2    14      11
5702        19        21.0          1    22      19
5703        19        18.0          5    11      12
5704        24        24.0          2     6      20
5705        29        29.0          3    16      56
...        ...         ...        ...  

In [5]:
# Import Pycaret library depending on the algorithm choice
pycaret = importlib.import_module(f"pycaret.{algorithm_choice}")

In [6]:
# Initialize data in PyCaret with all the defined parameters
pycaret.setup(data=train_data, target=target, session_id=123)

,Description,Value
0,Session id,123
1,Target,Bakerloo10
2,Target type,Regression
3,Original data shape,"(5701, 5)"
4,Transformed data shape,"(5701, 5)"
5,Transformed train set shape,"(3990, 5)"
6,Transformed test set shape,"(1711, 5)"
7,Numeric features,4
8,Preprocess,True
9,Imputation type,simple


<h3>6. Feature Engineering and Model Training</h3>

Here we are using the pycaret automl tool to train the model. The automl tool tries a number of machine learning algorithms depending on the type of machine learning problem you are trying to solve <br>
(regression, classification or time series). The automl tool then selects the best model based on the accuracy metrics of the tried models. 

In [7]:
# Train and evaluate the performance of all estimators available in the model library using cross-validation.
bestModel = pycaret.compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,0.7181,1.2056,1.0943,0.9794,0.0708,0.0521,0.1140
lightgbm,Light Gradient Boosting Machine,0.7430,1.3402,1.1501,0.9771,0.0797,0.0553,0.4360
xgboost,Extreme Gradient Boosting,0.7207,1.3435,1.1545,0.9771,0.0719,0.0491,0.0130
rf,Random Forest Regressor,0.6629,1.3632,1.1579,0.9767,0.0687,0.0402,0.0590
et,Extra Trees Regressor,0.6344,1.4357,1.1901,0.9754,0.0680,0.0375,0.0470
gbr,Gradient Boosting Regressor,0.8943,1.7672,1.3248,0.9696,0.0957,0.0697,0.0260
knn,K Neighbors Regressor,0.9110,1.9823,1.3997,0.9660,0.0872,0.0591,0.0070
dt,Decision Tree Regressor,0.7226,2.0739,1.4360,0.9644,0.0851,0.0429,0.0050
lar,Least Angle Regression,1.1633,2.7881,1.6649,0.9520,0.1383,0.1003,0.0040
br,Bayesian Ridge,1.1633,2.7881,1.6649,0.9520,0.1383,0.1003,0.0040


<h3>7. Model Evaluation: </h3>

Here we evaluate the performance of the best model, getting some visual representation of hyperparameters, features and other important details about the selected model.


In [9]:
# Evaluate model: Display UI analyzing Hyperparameters, Confusion Matrix, Class Report, etc.
pycaret.evaluate_model(bestModel)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

<h3>8. Saving Model for future predition</h3>

Here we are using a function that saves the model to s3. 

In [10]:
# Finalise model with validation data
final_model = pycaret.finalize_model(bestModel)


In [11]:
# save model locally
pycaret.save_model(final_model, model_name)

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['Bakerloo', 'dayOfWeek', 'hour',
                                              'minute'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=[],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('actual_estimator',
                  <catboost.core.CatBoostRegressor object at 0x2a5d4bd00>)]),
 'final_best_model.pkl')

In [12]:
# Upload model to s3
save_model_to_s3(model_name, f'{model_name}-model', final_model)

<h3>9. Deploying the model endpoints</h3> 

Here we use a function that creates the model endpoint in sagemaker. 

In [13]:
# deploy model to sagemaker endpoint
deploy_model(model_name, pycaret_ecr_name, instance_type, endpoint_name, role)

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/lanre.bakare/Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/lanre.bakare/Library/Application Support/sagemaker/config.yaml


ClientError: An error occurred (AccessDeniedException) when calling the CreateModel operation: User: arn:aws:sts::135544376709:assumed-role/AWSReservedSSO_AdministratorAccess_a474e246441df619/lanre.bakare@credera.co.uk is not authorized to perform: sagemaker:CreateModel on resource: arn:aws:sagemaker:eu-west-2:135544376709:model/mlops-pycaret-repo-2023-11-07-10-06-10-015 with an explicit deny in a service control policy

# Clean up stage
## Remove Endpoint and Endpoint Config

In [13]:
# Create a low-level SageMaker service client.
my_region = boto3.session.Session().region_name
sagemaker_client = boto3.client('sagemaker', region_name=my_region)

# Delete endpoint
sagemaker_client.delete_endpoint(EndpointName=endpoint_name)

# Delete endpoint configuration
sagemaker_client.delete_endpoint_config(EndpointConfigName=endpoint_name)

{'ResponseMetadata': {'RequestId': '3b9dc2e2-8025-49f1-a2f3-0e5d8526d097',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3b9dc2e2-8025-49f1-a2f3-0e5d8526d097',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Tue, 07 Nov 2023 09:15:10 GMT'},
  'RetryAttempts': 0}}